In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [2]:
df = pd.read_csv("../data/expo_impo_rubros_usos.csv",sep=",", encoding="latin-1")
df.valor = df.valor / 1000000 #En millones de USD
ultimo_anio = df.sort_values(["anio","mes"],ascending=True).anio.iloc[-1]
ultimo_mes = df.sort_values(["anio","mes"],ascending=True).mes.iloc[-1]

In [3]:
#Funciones para encontrar las tablas agrupadas

def genera_uso_rubro_mensual(df, comercio):
    valuacion = "Importaciones" if comercio == "impo" else "Exportaciones"
    df_separada = df[df.comercio == valuacion].reset_index(drop=True)
    df_separada["Participación porcentual"] = df_separada['valor'] / df_separada.groupby(["anio","mes"])['valor'].transform('sum')
    df_separada["Variación porcentual"] = df_separada.groupby(['rubro_uso'])["valor"].pct_change(12,fill_method=None)
    df_separada = df_separada[(df_separada.anio == ultimo_anio) & (df_separada.mes == ultimo_mes)].reset_index(drop = True)
    return df_separada

def genera_uso_rubro_acumulada(df, comercio):
    valuacion = "Importaciones" if comercio == "impo" else "Exportaciones"
    df_separada = df[df.comercio == valuacion].reset_index(drop=True)
    df_separada = df_separada[df_separada.mes <= ultimo_mes].reset_index(drop = True)
    df_separada = df_separada.groupby(["comercio","anio","rubro_uso"],as_index=False).sum()
    df_separada["Participación porcentual"] = df_separada['valor'] / df_separada.groupby(["anio"])['valor'].transform('sum')
    df_separada["Variación porcentual"] = df_separada.groupby(['rubro_uso'])["valor"].pct_change(1,fill_method=None)
    df_separada = df_separada[(df_separada.anio == ultimo_anio)].reset_index(drop = True)
    return df_separada

def genera_uso_rubro_agrupada():
    usos_mensual = genera_uso_rubro_mensual(df, "impo")
    rubros_mensual = genera_uso_rubro_mensual(df, "expo")
    df_mensual = pd.concat([usos_mensual, rubros_mensual]).reset_index(drop=True)
    usos_acumulada = genera_uso_rubro_acumulada(df, "impo")
    rubros_acumulada = genera_uso_rubro_acumulada(df, "expo")
    df_acumulada = pd.concat([usos_acumulada, rubros_acumulada]).reset_index(drop=True).drop("mes",axis=1)
    return df_mensual, df_acumulada

df_mensual, df_acumulada = genera_uso_rubro_agrupada()
df_acumulada

,comercio,anio,rubro_uso,valor,Participación porcentual,Variación porcentual
0,Importaciones,2022,Bienes de capital (BK),11581.759781,0.151409,0.285070
1,Importaciones,2022,Bienes de consumo (BC),7890.246971,0.103150,0.214434
2,Importaciones,2022,Bienes intermedios (BI),28160.712044,0.368147,0.199985
3,Importaciones,2022,Combustibles y lubricantes (CyL),12419.348375,0.162359,1.418387
4,Importaciones,2022,Piezas y accesorios para bienes de capital (PyA),14011.434447,0.183173,0.301761
5,Importaciones,2022,Resto,561.437307,0.007340,0.047629
6,Importaciones,2022,Vehículos automotores de pasajeros (VA),1868.148928,0.024422,0.200522
7,Exportaciones,2022,Combustibles y energía (CyE),7551.172719,0.091759,0.621483
8,Exportaciones,2022,Manufacturas de origen agropecuario (MOA),30766.793119,0.373866,0.075332
9,Exportaciones,2022,Manufacturas de origen industrial (MOI),21389.407598,0.259916,0.186799


In [4]:
DIC_MESES = {
    1:"enero",
    2:"febrero",
    3:"marzo",
    4:"abril",
    5:"mayo",
    6:"junio",
    7:"julio",
    8:"agosto",
    9:"septiembre",
    10:"octubre",
    11: "noviembre",
    12: "diciembre"
}


In [5]:
import plotly.express as px
fig = px.sunburst(df_acumulada, path=['comercio', 'rubro_uso'], values='valor', color='comercio',
                  hover_name = "comercio",
                  maxdepth=2,
                #   hover_data=
                  )
fig.update_layout(title_text = f"Millones de dólares comercializados según rubros y usos económicos<br>Acumulado hasta {DIC_MESES[ultimo_mes]} de {ultimo_anio}",
                  template = None, font_family = "verdana", separators = ",.")
fig.show()


c:\Users\mbasualdo\.conda\envs\indec\lib\site-packages\plotly\express\_core.py:1637: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_trees = df_all_trees.append(df_tree, ignore_index=True)
c:\Users\mbasualdo\.conda\envs\indec\lib\site-packages\plotly\express\_core.py:1637: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_trees = df_all_trees.append(df_tree, ignore_index=True)


In [6]:
df_acumulada.columns

Index(['comercio', 'anio', 'rubro_uso', 'valor', 'Participación porcentual',
       'Variación porcentual'],
      dtype='object')

In [20]:
def build_hierarchical_dataframe(df, levels, value_column, color_columns=None):
    """
    Build a hierarchy of levels for Sunburst or Treemap charts.

    Levels are given starting from the bottom to the top of the hierarchy,
    ie the last level corresponds to the root.
    """
    df_all_trees = pd.DataFrame(columns=['id', 'parent', 'value', 'color'])
    total_pa_color = df.valor.sum()
    for i, level in enumerate(levels):
        df_tree = pd.DataFrame(columns=['id', 'parent', 'value', 'color'])
        dfg = df.groupby(levels[i:]).sum()
        dfg = dfg.reset_index()
        df_tree['id'] = dfg[level].copy()
        if i < len(levels) - 1:
            df_tree['parent'] = dfg[levels[i+1]].copy()
        else:
            df_tree['parent'] = 'ICA'
        df_tree['value'] = dfg[value_column]
        
        # df_tree['color'] = dfg[color_columns]/total_pa_color
        df_all_trees = df_all_trees.append(df_tree, ignore_index=True)
        # df_all_trees = pd.concat([df_all_trees,df_tree] ,ignore_index=True)
    total = pd.Series(dict(id='ICA', parent='',
                              value=df[value_column].sum(),
                            #   color=df[color_columns]
                            ))
    df_all_trees = df_all_trees.append(total, ignore_index=True)
    #para reemplazar el nan del total
    # df_all_trees.color.iloc[-1] = 1
    # df_all_trees =pd.concat([df_all_trees,total], ignore_index=True)
    return df_all_trees

def plot_sunburst_socios(df_impo, acumulado:bool):
    fecha = F"acumulado hasta {DIC_MESES[ultimo_mes]}" if acumulado else F"mes de {DIC_MESES[ultimo_mes]}"
    socios_sas_mensual_tree = build_hierarchical_dataframe(df=df_impo, levels=["rubro_uso", "comercio"],value_column="valor", color_columns = "valor")

    fig = go.Figure(go.Sunburst(
        labels=socios_sas_mensual_tree['id'],
        parents=socios_sas_mensual_tree['parent'],
        values=socios_sas_mensual_tree['value'],
        branchvalues='total',
        marker=dict(
            # colors=socios_sas_mensual_tree['color'],
            # colorscale='Sunsetdark',
            # cmid=average_score
            ),
        hovertemplate='<b>%{label} </b> <br> CIF: $%{value:,.2f}<br> Proporcion: %{color:.2%}', #Me jode la proporcion, sí o sí entra por color
        name='',
        maxdepth=2
        ))
    fig.update_layout(title_text = f"Principales socios comerciales {fecha}", template = None,
                    margin = {"t":70, "b":20, "l":10, "r":10},separators = ",." ,
                    #   uniformtext=dict(minsize=10, mode='hide'),
                    font_family = "verdana",
                    # paper_bgcolor='rgba(0,0,0,0)',
                    ) 
    
    return fig

plot_sunburst_socios(df_acumulada, acumulado=True)
plot_sunburst_socios(df_mensual, acumulado=False)

C:\Users\mbasualdo\AppData\Local\Temp\ipykernel_8040\3576934485.py:22: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\mbasualdo\AppData\Local\Temp\ipykernel_8040\3576934485.py:22: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\mbasualdo\AppData\Local\Temp\ipykernel_8040\3576934485.py:28: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\mbasualdo\AppData\Local\Temp\ipykernel_8040\3576934485.py:22: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\mbasualdo\AppData\Local\Temp\ipykernel_8040\3576934485.py:22: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version